In [ ]:
%cd ~
import matplotlib
matplotlib.use("Agg")
import pyfits,glob,time,scipy
import scipy.interpolate
from scipy.special import erf
from pylab import *
from subprocess import call
from scipy.optimize import fmin_powell

topdir='/sandbox/lsst/lsst/GUI/'
thedir=topdir+'notebooks/'
datadir=topdir+'/20170821_002_spots/'
%cd $thedir

configfile=topdir+'sextractor/default-array_dither.sex'
paramfile=topdir+'sextractor/default-array_dither.param'
maskdir=topdir+'sextractor/masks/'


In [ ]:
def remove_overscan_xy(image,x_overscan_start,x_overscan_end,y_overscan_start,y_overscan_end):
    overscan=image[:y_overscan_start,x_overscan_start+1:x_overscan_end]
    image=image[:y_overscan_start,:x_overscan_start]
    finalimg=image-matrix(median(overscan,axis=1)).T*np.ones(shape(image)[1])
    return array(finalimg)

def make_reg_from_ldac(cat_ldac_file,text_tag):
    thecat=pyfits.getdata(cat_ldac_file,'LDAC_OBJECTS')
    f = open(cat_ldac_file+'.reg','w')
    for i in range(len(thecat)):
        xcoo,ycoo=thecat['XWIN_IMAGE'][i],thecat['YWIN_IMAGE'][i]
        r=thecat['A_IMAGE'][i]
        thetext=thecat[text_tag][i]
        f.write('circle '+str(xcoo)+' '+str(ycoo)+' '+str(r)+'#text="'+str(thetext)+'"\n')
    f.close()
    
def Area(xl, xh, yl, yh, sigmax, sigmay, Imax):
    # Calculates how much of a 2D Gaussian falls within a rectangular box
    ssigx = sqrt(2) * sigmax
    ssigy = sqrt(2) * sigmay    
    I = (erf(xh/ssigx)-erf(xl/ssigx))*(erf(yh/ssigy)-erf(yl/ssigy))
    return Imax * I / 4.0

In [ ]:
zfilelist=sort(glob.glob(datadir+'ITL-3800C-002_spot_spot_5??_20170821??????_ct.fits')
               +glob.glob(datadir+'ITL-3800C-002_spot_spot_6??_20170821??????_ct.fits')
               +glob.glob(datadir+'ITL-3800C-002_spot_spot_7??_20170821??????_ct.fits'))
print len(zfilelist)
#zfilelist

In [ ]:
for fitsfilename in zfilelist[100:250]: 
    tfile1=time.time() 
    for i in [8]:
        tstart=time.time() 
        extname=pyfits.getheader(fitsfilename,i)['EXTNAME'] 
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        # cut off the overscan 
        outname=fitsfilename[:-5]+extname+'.fits' 
        pyfits.writeto(outname,overscansubimg,clobber=True) 
        test=call(["sex",outname,"-c",configfile,"-CATALOG_NAME",outname+'.cat']) 
        make_reg_from_ldac(outname+'.cat','NUMBER') 
        tend=time.time() 
        print extname+" time: "+str(tend-tstart)[:4] 
        print "Time taken for file "+str(fitsfilename[-26:-23])+": "+str(time.time()-tfile1)

#### Using the sextractor catalogs produced above to make a map of the sextractor measurement named below

In [ ]:
class Array2dSet:
    def __init__(self,xmin,xmax,nx,ymin,ymax,ny):
        # This packages up a set of postage stamp images,
        # each image of which is nx * ny pixels
        self.nx=nx
        self.ny=ny
        self.nstamps=0

        self.xmin=xmin
        self.ymin=ymin
        
        self.xmax=xmax
        self.ymax=ymax
        
        self.dx=(xmax-xmin)/nx
        self.dy=(ymax-ymin)/ny
        
        self.x=linspace(xmin+self.dx/2,xmax-self.dx/2,nx)
        self.y=linspace(ymin+self.dy/2,ymax-self.dy/2,ny)

        self.xint = zeros([self.nstamps],dtype = int)
        self.yint = zeros([self.nstamps],dtype = int)
        self.data=zeros([nx,ny,self.nstamps])
        self.xoffset=zeros([self.nstamps])
        self.yoffset=zeros([self.nstamps])
        self.imax=zeros([self.nstamps])
        return

    def AddSpot(self,xint, yint, xoff, yoff, stamp):
        # This adds a stamp to an existing spotlist
        self.nstamps += 1
        self.xint = append(self.xint, xint)
        self.yint = append(self.yint, yint)
        self.xoffset = append(self.xoffset, xoff)
        self.yoffset = append(self.yoffset, yoff)
        self.imax = append(self.imax, 0.0)
        self.data = dstack((self.data, stamp))
        return


In [ ]:
def BuildSpotLists(zfilelist, segmentnumber, nx, ny, minsize, maxsize):
    stampxmin = -(int(nx/2)+0.5)
    stampxmax = -stampxmin
    stampymin = -(int(ny/2)+0.5)
    stampymax = -stampymin
    dx=(stampxmax-stampxmin)/nx
    dy=(stampymax-stampymin)/ny
        
    x=linspace(stampxmin+dx/2,stampxmax-dx/2,nx)
    y=linspace(stampymin+dy/2,stampymax-dy/2,ny)

    #xcoomin = 200
    #xcoomax = 250
    #ycoomin = 1850
    #ycoomax = 1900
    xcoomin = 50
    xcoomax = 450
    ycoomin = 50
    ycoomax = 1950
    spotlists = []
    spotdata = zeros([nx,ny])

    for n,fitsfilename in enumerate(zfilelist):
        hdr=pyfits.getheader(fitsfilename,segmentnumber)
        extname = hdr['EXTNAME']
        img=pyfits.getdata(fitsfilename,extname) 
        overscansubimg=remove_overscan_xy(img,509,542,2000,2022) 
        catname=fitsfilename[:-5]+extname+'.fits.cat.reg' 
        catfile = open(catname,'r')
        catlines = catfile.readlines()
        catfile.close()
        for line in catlines:
            try:
                size = float(line.split()[3].split('#')[0])
                if size < minsize or size > maxsize:
                    continue
                xcoord = float(line.split()[1])
                ycoord = float(line.split()[2])
                if xcoord < xcoomin or xcoord > xcoomax or ycoord < ycoomin or ycoord > ycoomax:
                    continue
                xint = int(xcoord-0.5)
                yint = int(ycoord-0.5)
                xmin = xint - int(nx/2)
                xmax = xint + int(nx/2) + 1
                ymin = yint - int(ny/2)
                ymax = yint + int(ny/2) + 1
                stamp = overscansubimg[ymin:ymax, xmin:xmax]
                xsum = 0.0
                ysum = 0.0
                datasum = 0.0
                for i in range(nx):
                    for j in range(ny):
                        spotdata[i,j] = float(stamp[j,i])                    
                        ysum += y[j] * spotdata[i,j]
                        xsum += x[i] * spotdata[i,j]
                        datasum += spotdata[i,j]
                xoff = xsum / datasum
                yoff = ysum / datasum
                if n == 0:
                    spotlists.append(Array2dSet(stampxmin,stampxmax,nx,stampymin,stampymax,ny))
                    spotlists[-1].AddSpot(xint,yint,xoff,yoff,spotdata)
                else:
                    for spotlist in spotlists:
                        if abs(spotlist.xint[-1] - xint) <= 3 and abs(spotlist.yint[-1] - yint) <= 3:
                            spotlist.AddSpot(xint,yint,xoff,yoff,spotdata)
                            break
                        # If not found, add dummy data to keep list ordering
                        spotdata = zeros([nx,ny])
                        spotlist.AddSpot(0,0,0.0,0.0,spotdata)
            except:
                continue
    return spotlists

In [ ]:
starttime = time.time()
spotlists = BuildSpotLists(zfilelist, 8, 9, 9, 0.7, 1.4)
print "Elapsed time = ", time.time() - starttime

In [ ]:
for i in range(NumFluxes):
    print len(fullspotlists[i])
print fullspotlists[3][34].nstamps
print fullspotlists[2][37].data[:,:,3]


In [ ]:
import forward

def FOM(params):
    [sigmax, sigmay] = params
    result = forward.forward(spotlist,sigmax,sigmay)
    return result

def PyFOM(params):
    fom = 0.0
    [Imax, sigmax, sigmay] = params
    area=zeros([spotlist.nx,spotlist.ny])
   
    for spot in range(spotlist.nstamps):
        for ii in range(spotlist.nx):
            for jj in range(spotlist.ny):
                xl = spotlist.x[ii] - spotlist.xoffset[spot] - 0.5
                xh = xl + 1.0
                yl = spotlist.y[jj] - spotlist.yoffset[spot] - 0.5
                yh = yl + 1.0
                area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax)
                fom += square(area[ii,jj]-spotlist.data[ii,jj,spot])
    return fom


In [ ]:
for i, segmentnumber in enumerate([8]):
    hdr=pyfits.getheader(zfilelist[0],segmentnumber)
    extname = hdr['EXTNAME']
    print "Segment # %d is %s"%(segmentnumber,extname)


In [ ]:
nx = 9
ny = 9
figure()
spot = 228
flux = 3
spotlist = fullspotlists[flux][spot]
frame = 43
print "nstamps = %d"%spotlist.nstamps
param0 = [1.00, 1.00]
args = ()
Result = fmin_powell(FOM, param0, args)
print Result
sigmax = abs(Result[0])
sigmay = abs(Result[1])
ADU_correction = Area(-0.5,0.5,-0.5,0.5,sigmax, sigmay,1.0)
Imax = spotlist.imax[frame] 
suptitle=("Typical Fit, Spot = %d"%spot)
area=zeros([nx,ny])
for ii in range(nx):
    for jj in range(ny):
        xl = ii - int(nx/2) - spotlist.xoffset[frame] - 0.5
        xh = xl + 1.0
        yl = jj - int(ny/2) - spotlist.yoffset[frame] - 0.5
        yh = yl + 1.0
        area[ii,jj] = Area(xl, xh, yl, yh, sigmax, sigmay, Imax) 
subplots_adjust(wspace = 2.0)
subplot(1,3,1)
imshow(spotlist.data[:,:,frame] ,interpolation="None")
subplot(1,3,2)
plot(spotlist.data[int(nx/2),:,frame] , lw = 2, label="Data")
plot(area[int(nx/2),:], lw = 2, label="Model")
xlabel("X (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
subplot(1,3,3)
plot(spotlist.data[:,int(ny/2),frame], lw = 2, label="Data")
plot(area[:,int(ny/2)], lw = 2, label="Model")
xlabel("Y (Pixels)")
ylabel("Signal(ADU)")
xticks([0,2,4,6,8])
legend(loc = (-6.5,0.8))
savefig(datadir+"Typical_Fit_22Aug17_%d_%d_%d"%(flux,spot,frame))




In [ ]:
starttime = time.time()
NumSpots = 1696
xints = zeros([NumFluxes, NumSpots])
yints = zeros([NumFluxes, NumSpots])
sigxs = zeros([NumFluxes, NumSpots])
sigys = zeros([NumFluxes, NumSpots])
imaxs = zeros([NumFluxes, NumSpots])
for i in range(NumFluxes):
    for j, spotlist in enumerate(fullspotlists[i]):
        param0 = [1.00, 1.00]
        args = ()
        Result = fmin_powell(FOM, param0, args)
        sigxs[i,j] = abs(Result[0])
        sigys[i,j] = abs(Result[1])
        xints[i,j] = spotlist.xint[0]
        yints[i,j] = spotlist.yint[0]
        imaxs[i,j] = spotlist.imax.mean()

print "Elapsed time = ", time.time() - starttime

In [ ]:
print xints.shape, yints.shape, sigxs.shape, sigys.shape, imaxs.shape
print mean(imaxs, axis=1)
print mean(sigys, axis=1)
print xints[:,-20], yints[:,-20]
print sigxs[:,0], sigys[:,0]

In [ ]:
for i in range(20):
    print "(%d,%d),(%d,%d),(%d,%d),(%d,%d),(%d,%d)"%(xints[0,i],yints[0,i],xints[1,i],yints[1,i],xints[2,i],yints[2,i],xints[3,i],yints[3,i],xints[4,i],yints[4,i])

In [ ]:
print "(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f)"%(sigxs[0,3],sigys[0,3],sigxs[1,5],sigys[1,5],sigxs[2,0],sigys[2,0],sigxs[3,6],sigys[3,6],sigxs[4,5],sigys[4,5])
print "(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f)"%(sigxs[0,2],sigys[0,2],sigxs[1,8],sigys[1,8],sigxs[2,1],sigys[2,1],sigxs[3,7],sigys[3,7],sigxs[4,6],sigys[4,6])
print "(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f),(%.4f,%.4f)"%(sigxs[0,4],sigys[0,4],sigxs[1,7],sigys[1,7],sigxs[2,2],sigys[2,2],sigxs[3,8],sigys[3,8],sigxs[4,8],sigys[4,8])

In [ ]:
for i in range(-20,-1):
    print "(%d,%d)"%(fullspotlists[0][0].xint[i],fullspotlists[0][0].yint[i])

In [ ]:
figure()
subplot(1,1,1,aspect=1)
scatter(xints, yints)
savefig(datadir+"Test.png")

In [ ]:
from matplotlib.mlab import griddata
fig = figure()
x = array(xints)
y = array(yints)
zx = array(sigxs)
zy = array(sigys)
xi = np.linspace(x.min()-1, x.max()+1, 100)
yi = np.linspace(y.min()-1, y.max()+1, 100)
zxi = griddata(x, y, zx, xi, yi)#, interp='linear')
zyi = griddata(x, y, zy, xi, yi)#, interp='linear')
subplot(1,2,1,aspect = 1)
title("Sigma-x")
contourf(xi, yi, zxi)
colorbar()
subplot(1,2,2,aspect = 1)
title("Sigma-y")
contourf(xi, yi, zyi)
colorbar()
savefig(datadir+"Test2.png")

